In [1]:
from sklearn.ensemble import ExtraTreesClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import gc

In [2]:
dt_train = pd.read_csv("../../data/MelbDatathon2017/csv/dt_train_eng_all.csv")
dt_test = pd.read_csv("../../data/MelbDatathon2017/csv/dt_test_eng_all.csv")

In [3]:
y = dt_train.pop("Target")
X = dt_train

In [4]:
y_test = dt_test.pop("Target")
X_test = dt_test
X_test = X_test.loc[:, ~X_test.columns.str.contains('^Unnamed')]

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.8, random_state = 42)

In [6]:
id_train = X_train.pop("Patient_ID")
id_valid = X_valid.pop("Patient_ID")
id_test = X_test.pop("Patient_ID")

In [7]:
X_train.shape

(55840, 2310)

In [8]:
del X, y, dt_train, dt_test; gc.collect()

78

In [15]:
model_extra = ExtraTreesClassifier(n_jobs = 7, n_estimators = 1500
                                   , max_depth = 18, min_samples_split = 3, random_state = 888, verbose = 0)
model_extra.fit(X_train, y_train)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=18, max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=3, min_weight_fraction_leaf=0.0,
           n_estimators=1500, n_jobs=7, oob_score=False, random_state=888,
           verbose=0, warm_start=False)

In [26]:
preds_valid = model_extra.predict_proba(X_valid)[:, 1]

NameError: name 'X_valid' is not defined

In [17]:
roc_auc_score(y_valid, preds_valid)

0.96708415910456069

In [24]:
del X_train, y_train, X_valid, y_valid; gc.collect()

0

In [27]:
preds_test = model_extra.predict_proba(X_test)[:, 1]

In [28]:
dt_submit = pd.DataFrame({"Patient_ID": id_test, "Diabetes": preds_test})

In [29]:
dt_submit[["Patient_ID", "Diabetes"]].to_csv("../../data/MelbDatathon2017/submission/extraTree_2_tuned.csv", index = False)